This R/Bioconductor notebook is the first of 2 parts for the analysis of human data presented in Figure 6. This notebook ingests the location of ChIP-seq peaks (from A. Soufi, personal correspondence), and outputs various intermediate files for further processing in FIMO and by my own scripts.

# Initialization

In [1]:
# for my sanity - map some function names that I think exist to what actually exist
nrows <- nrow
ncols <- ncol
len <- length

In [2]:
library(repr)
options(repr.plot.width=6, repr.plot.height=3)

In [3]:
library(tracktables)
library(rtracklayer)
library(GenomicFeatures)
library(BSgenome.Hsapiens.UCSC.hg38)

Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Loading requi

In [4]:
# get seqinfo (but also since the BSgenome is from UCSC, remap coordinates back to NCBI/ensembl)

hg38info <- keepStandardChromosomes(seqinfo(BSgenome.Hsapiens.UCSC.hg38), pruning.mode = "tidy")
newStyle <- mapSeqlevels(seqlevels(hg38info),"NCBI")
hg38info <- (renameSeqlevels(hg38info, newStyle))
hg38info

Warning message in `seqlevels<-`(`*tmp*`, pruning.mode = pruning.mode, value = c("chr1", :
“'pruning.mode' is ignored in "seqlevels<-" method for Seqinfo objects”

Seqinfo object with 25 sequences (1 circular) from hg38 genome:
  seqnames seqlengths isCircular genome
  1         248956422      FALSE   hg38
  2         242193529      FALSE   hg38
  3         198295559      FALSE   hg38
  4         190214555      FALSE   hg38
  5         181538259      FALSE   hg38
  ...             ...        ...    ...
  21         46709983      FALSE   hg38
  22         50818468      FALSE   hg38
  X         156040895      FALSE   hg38
  Y          57227415      FALSE   hg38
  MT            16569       TRUE   hg38

# ChIP-peaks (from Soufi *et al*)
(from **Soufi 2015** (personal correspondence)):

Peak locations were extracted and then lifted over via UCSC `liftOver` from hg18 (EIGHTEEN) to GRCh38/hg38. These locations are available on request.

In [5]:
SOX_peaks <- import("~/SoxOct/Soufi2015/Soufi2015_S_sites.hg38.bed")
OCT_peaks <- import("~/SoxOct/Soufi2015/Soufi2015_O_sites.hg38.bed")

SOX_peaks <- keepStandardChromosomes(SOX_peaks, pruning.mode = 'coarse')
OCT_peaks <- keepStandardChromosomes(OCT_peaks, pruning.mode = 'coarse')

In [6]:
# note that Soufi peaks are in UCSC coordinates:
SOX_peaks <- renameSeqlevels(SOX_peaks, mapSeqlevels(seqlevels(SOX_peaks), "NCBI"))
OCT_peaks <- renameSeqlevels(OCT_peaks, mapSeqlevels(seqlevels(OCT_peaks), "NCBI"))

In [7]:
# drop MT chromosome
SOX_peaks <- SOX_peaks[seqnames(SOX_peaks) != "MT"]
OCT_peaks <- OCT_peaks[seqnames(OCT_peaks) != "MT"]

# manually add names since Soufi et al BED doesn't have any
SOX_peaks$name <- paste0("Soufi_SOX_peak_", 1:len(SOX_peaks))
OCT_peaks$name <- paste0("Soufi_OCT_peak_", 1:len(OCT_peaks))

In [8]:
# add a dummy MT seqlevel
seqlevels(SOX_peaks) <- c(seqlevels(SOX_peaks), "MT")
seqlevels(OCT_peaks) <- c(seqlevels(OCT_peaks), "MT")

seqlevels(OCT_peaks) <- c(seqlevels(OCT_peaks), "Y") # only Oct is missing Y inexplicably

In [9]:
# sort the seqlevels... because we have to?
SOX_peaks <- sortSeqlevels(SOX_peaks)
OCT_peaks <- sortSeqlevels(OCT_peaks)

In [10]:
seqinfo(SOX_peaks) <- hg38info
seqinfo(OCT_peaks) <- hg38info

# DNA export

I export the genomic DNA sequences to use `FIMO` to identify sequences conforming to given binding motifs.

In [11]:
writeDNAtofile <- function(grange, path){
    # map the seqlevels back to UCSC
    grforexport <- renameSeqlevels(grange, mapSeqlevels(seqlevels(grange), "UCSC"))
    print(seqlevels(grforexport))

    # add flanking 200 bp
    #DNAflank <- 200
    DNAflank <- 0 # since it's already a 200bp-wide thing
    print(paste0("flanking DNA: ", DNAflank))
    grforexport <- grforexport + DNAflank
    
    grforexport <- trim(grforexport)
    
    DNAforexport <- getSeq(BSgenome.Hsapiens.UCSC.hg38, names = grforexport) #actual sequence lookup step
    names(DNAforexport) <- mcols(grforexport)$name # add names so I know what it is
    
    print(DNAforexport)
    
    # write it out to disk
    writeXStringSet(DNAforexport, filepath = path)
}

In [12]:
writeDNAtofile(SOX_peaks, "~/SoxOct/public/Soufi2015/Sox_peakseqs_200.fasta")

 [1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"  "chrY"  "chrM" 
[1] "flanking DNA: 0"
  A DNAStringSet instance of length 32884
        width seq                                           names               
    [1]   199 CTATGTCTCTGAACTATTAAT...GTATCCATGGTCTGAAACAAT Soufi_SOX_peak_1
    [2]   199 ATGTATTTGGAAATGACAAAA...GTTTTGAAAAGCAATGAAAGG Soufi_SOX_peak_2
    [3]   199 AGCGTTTGATAAAATCACAAA...TAGTTAAATAAAAAACACAAC Soufi_SOX_peak_3
    [4]   199 CATTAAATGTAATAACCTATG...ACAGAGAGAAGAAGATAAAGC Soufi_SOX_peak_4
    [5]   199 GACTATATACTACTTTTTATT...GAACATGCTTGGAGTTCTTTG Soufi_SOX_peak_5
    ...   ... ...
[32880]   199 AAACAGACAACTCAACAAAAT...GGCTCAAAGACAGGTCATTTG Soufi_SOX_peak_32880
[32881]   199 TAGTTAAGTCCTTACCAATCG...ATCCCACACCAATGATAACAA Soufi_SOX_peak_32881
[32882]   199 GATGGCTGTAAATGTGTGGCT...TCCAGCTTTGTTCTTTTTTGC Soufi_SOX_peak_

In [13]:
writeDNAtofile(OCT_peaks, "~/SoxOct/public/Soufi2015/Oct_peakseqs_200.fasta")

 [1] "chr1"  "chr2"  "chr3"  "chr4"  "chr5"  "chr6"  "chr7"  "chr8"  "chr9" 
[10] "chr10" "chr11" "chr12" "chr13" "chr14" "chr15" "chr16" "chr17" "chr18"
[19] "chr19" "chr20" "chr21" "chr22" "chrX"  "chrY"  "chrM" 
[1] "flanking DNA: 0"
  A DNAStringSet instance of length 19111
        width seq                                           names               
    [1]   199 ATGTGCACACATATAAACACT...CACACATATGCACATGATATA Soufi_OCT_peak_1
    [2]   199 GGGAAAGAATAATGCGTTTGC...TTCAAGCAAGTCCTTTGAGAA Soufi_OCT_peak_2
    [3]   199 TGCACAGATGAGATTTCATCC...CAATGCACAGTGAATCTGCTT Soufi_OCT_peak_3
    [4]   199 TCACAACAAAGCTCTTTGTCT...TAATTAGATCTGCATTTTAGA Soufi_OCT_peak_4
    [5]   199 ATGTATATGCATATGTGTATA...TGTATATGCATATGCATGTGC Soufi_OCT_peak_5
    ...   ... ...
[19107]   199 CTCAGAGAGGCCTTCCCTAAC...AGCATTTTGCCTTCGTATTTA Soufi_OCT_peak_19107
[19108]   199 GGCTAGGGGCGTTCCACCAGG...TAAAGTTTGAGGCACCAACAG Soufi_OCT_peak_19108
[19109]   545 CTATCCCTTCTCTTCACACGC...GGGAGGGGTTGGGAAACTATT Soufi_OCT_peak_

# CHIP-confirmed motif locations in genome (via FIMO)

I use `FIMO` with `--verbosity 3` and `--thresh .001` to identify sites conforming to the canonical Sox2 motif (MA0143.3) or the Sox2 74% noncanonical motif in Sox2 peak sequences from above. I similarly use `FIMO` to identify canonical Oct4 (MA1115.1) and noncanonical (42% anad 28% variants) in Oct4 peak sequences from above.

I then use [`py3_motif-matching-human-public.ipynb`](py3_motif-matching-human-public.ipynb) to convert the coordinates from FIMO back into genomic coordinates. This requires that I have the original peak locations as a GTF.

In [14]:
export(SOX_peaks, '~/SoxOct/public/Soufi2015/Sox_peaks_200.gtf')
export(OCT_peaks, '~/SoxOct/public/Soufi2015/Oct_peaks_200.gtf')